# Demonstrate Tensorflow Flow Lite

Training a Model on Computer, Optimizing the model with TF Lite and deploy on a mobile device

## Model Optimization Process

- Optimize Model Topology (Decrease number of layers, faster operations, less fully connected layers, smaller etc). But this approach sacrifices accuracy over speed

- Network Parameters precision reduction with quantization. Quantization is a technique where we reduce the precision spaces after the floating numbers. In most cases, this technique wont affect the model accuracy.
    - Pre Training Quantization
    - Post training Quantization

## Library Imports

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

print(f"Tensorflow version used - {tf.__version__}")

Tensorflow version used - 2.9.1


## Dataset Preprocessing

### Loading the FashionMNIST dataset

In [2]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


### Image Normalization

In [3]:
X_train = X_train/255.0
X_test = X_test/255.0

### Reshaping of the dataset

In [4]:
# Check the shape of the training features
X_train.shape

(60000, 28, 28)

In [5]:
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

print(f"Reshaped training features - {X_train.shape}")

Reshaped training features - (60000, 784)


## Building the model

In [6]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=128, activation=tf.keras.activations.relu, input_shape=(784, )))
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(tf.keras.layers.Dense(units=10, activation=tf.keras.activations.softmax))

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

# Check the overall model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               100480    
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
_________________________________________________________________


## Training the Model

In [7]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 8s 2ms/step - loss: 0.5327 - sparse_categorical_accuracy: 0.8104
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.4034 - sparse_categorical_accuracy: 0.8518
Epoch 3/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3680 - sparse_categorical_accuracy: 0.8648
Epoch 4/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3445 - sparse_categorical_accuracy: 0.8724
Epoch 5/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3309 - sparse_categorical_accuracy: 0.8773
Epoch 6/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3154 - sparse_categorical_accuracy: 0.8848
Epoch 7/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3101 - sparse_categorical_accuracy: 0.8845
Epoch 8/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2985 - sparse_categorical_accuracy: 0.8886
Epoch 9/10
1875/1875 [==========

## Evaluating the Model

In [8]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy - {test_accuracy}")

313/313 [==============================] - 1s 2ms/step - loss: 0.3414 - sparse_categorical_accuracy: 0.8807
Test Accuracy - 0.8806999921798706


## Converting the Model into Tensorflow Lite Version

### Saving the Model

In [9]:
model_name = 'fashion_mobile_model.h5'
tf.keras.models.save_model(model, model_name)

### Creating the TFLite Converter

In [11]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)

### Converting the model

In [12]:
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\SuvoB\AppData\Local\Temp\tmpm5qfrn41\assets


### Saving the TFLite version of the model

In [13]:
with open("tf_model.tflite", "wb") as f:
    f.write(tflite_model)